# Capella API: Search, Order, and Download Tutorial

In [ ]:
# Required libraries:
# requests
# json
# folium
# urllib

In [ ]:
!pip install folium

In [ ]:
# Your username and password must be saved in a .json file named 'credentials.json' and formatted as follows.
# {"username": "yourusername","password": "xxxxxxxxx"}

In [ ]:
import requests
import json

# Helper function to printformatted JSON using the json module
def p(data):
    print(json.dumps(data, indent=2))

# Capella PROD API endpoints
URL = 'https://api.data.capellaspace.com'
token = '/token'
collections = '/catalog/collections'
catsearch = '/catalog/stac/search'
orders = '/orders/'

#Load my username and password

with open('credentials.json') as f:
    data = json.load(f)
    username = data['username']
    password = data['password']

In [ ]:
#Get a valid token from the auth service, access tokens are valid for 15 minutes
r = requests.post(URL + token, 
                  headers = {'Content-Type': 'application/x-www-form-urlencoded'}, auth=(username,password))

accesstoken = r.json()["accessToken"]

#print the token
#print("Access Token: " + accesstoken)

headers = {'Authorization':'Bearer ' + accesstoken}

In [ ]:
#See what collections are available, currently there are four collections available
r = requests.get(URL + collections, headers=headers)
print(URL + collections)
p(r.json())

In [ ]:
# Post search filters
filters = {
  "bbox": [-121.1298,41.3422,-119.4747,42.1484], # lower left coodinate and upper right coordinate, in decimal degrees
  "time": "2019-01-01T00:00:00Z/2019-12-31T12:31:12Z", # start date/end date and time in YYYY-MM-DDTHH:MM:SSZ format
  "limit": 50, # overwrite the default pagination limit of 10, adjust as necessary
  "query": {"sar:instrument_mode": {"eq": "IW"}}, # add query parameters to filter search results
#            "sar:pass_direction": {"eq": "ascending"}, 
#            "sar:type": {"eq":"GRD"}},
  "sort": [{"field": "dtr:start_datetime","direction": "desc"}],
}
headers = {'Content-Type': 'application/json',
  'Accept': 'application/geo+json', 'Authorization':'Bearer ' + accesstoken}
r = requests.post(URL + catsearch, json=filters, headers=headers)

# Inspect the results
p(r.json())

In [ ]:
# Display the results on a folium map
import folium
loc = [(filters["bbox"][1]+filters["bbox"][3])/2, (filters["bbox"][0]+filters["bbox"][2])/2] 
#print(loc)

m = folium.Map(location=[32, -118],zoom_start=4)
folium.GeoJson(
    r.text,
).add_to(m)

m

In [ ]:
# Make an Order
features = r.json()["features"]
granulelist = []

# Loop over all the features from the response and add to an array for an order
for f in features:
    item = {"CollectionId": f["collection"], "GranuleId": f["id"]}
    granulelist.append(item)
    
myorder = {"Items": granulelist}

# Post the order and inspect the result
r = requests.post(URL + orders, json=myorder, headers=headers)
p(r.json())

In [ ]:
# Get the STAC records with the signed URLs using the /download endpoint
myorderid = r.json()["orderId"]
r = requests.get(URL + orders + myorderid + '/download', headers=headers)
p(r.json())

In [ ]:
# Download the results
features = r.json()
     
basefp = 'C:/data/SAR/test/' # Local directory #to save data

for feature in features:
           
    filepath = feature["assets"]["VH-iw1"]["href"] # the second nested dictionary ("VH-iw1" here) must be changed for different assets
    # e.g. filepath = feature["assets"]["VH-iw1-metadata"]["href"] will return the url for the assets .xml metadata file
    #print(filepath)

    filename = filepath[filepath.rfind("/")+1:]
    #print(filename)
    sep = "?"
    truncname = filename.split(sep, 1)[0]
    #print(truncname)
    
    outfp = basefp + truncname
    
    import urllib
    f = urllib.request.urlretrieve(filepath, outfp)
